In [73]:
import os
import sys
import urllib, io

import numpy as np
import scipy.stats as stats
import pandas as pd

import pymongo as pm
from collections import Counter
import json
import re
import ast

from PIL import Image, ImageOps, ImageDraw, ImageFont 

from io import BytesIO
import base64

import  matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

import seaborn as sns
sns.set_context('talk')
sns.set_style('darkgrid')

from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

In [74]:
## directory & file hierarchy
proj_dir = os.path.abspath('..')
datavol_dir = os.path.join(proj_dir,'data')
analysis_dir = os.path.abspath(os.path.join(os.getcwd(),'..'))
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
json_dir = os.path.join(results_dir,'json')
exp_dir = os.path.abspath(os.path.join(proj_dir,'experiments'))
png_dir = os.path.abspath(os.path.join(datavol_dir,'png'))
jefan_dir = os.path.join(analysis_dir,'jefan')
will_dir = os.path.join(analysis_dir,'will')

## add helpers to python path
if os.path.join(proj_dir,'stimuli') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'stimuli'))
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)       

In [75]:
## set vars 
auth = pd.read_csv(os.path.join(analysis_dir,'auth.txt'), header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'cogtoolslab.org' ## cocolab ip address

## have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['block_construction']
coll = db['silhouette']

## which iteration name should we use?
iterationName = 'Exp2Pilot3'


## Sanity Checks

In [31]:
# Ensure one to one gameID and workerId 
# Should only happen if a repeat worker gets through

query = coll.find({"$and":[
                        {'workerId':{'$exists':True}},
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'trial_end'},
                        {"$or":[{'iterationName':'pilot2'},
                                {'iterationName':'pilot3'},
                                {'iterationName':'pilot4'},
                                {'iterationName':'Exp2Pilot1'}]},
                        {'trialNum':0}]
                     })

df_trial_end_full = pd.DataFrame(list(query.sort('timeAbsolute')))
#df_trial_end_full[['workerId','gameID']]


assert (np.mean(df_trial_end_full['workerId'].value_counts()) == np.mean(df_trial_end_full['gameID'].value_counts()))

### Find full datasets for Silhouette_1

In [34]:
query = coll.find({"$and":[
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'trial_end'},
                        {'iterationName': iterationName}]
                     })
complete_data_df = pd.DataFrame(query)
#complete_data_ids = list(complete_data_df['workerId'])

In [35]:
complete_data_df['workerId'].nunique()

KeyError: 'workerId'

In [36]:
complete_data_df

""


## Collect data from db and filter with sanity checks

In [ ]:
query = coll.find({"$and":[
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'trial_end'},
                        {'iterationName':iterationName}]
                     })

df_trial_end_full = pd.DataFrame(list(query.sort('timeAbsolute')))



# filter dataframe for complete datasets
df_trial_end_full_filtered = df_trial_end_full[df_trial_end_full.workerId.isin(complete_data_ids)]


# reduce to crucial information
df_trial_end_reduced_filtered = df_trial_end_full_filtered[['gameID','trialNum','phase','condition',
                                                            'eventType','score','normedScore','numBlocks',
                                                            'timeAbsolute','timeRelative','buildTime',
                                                            'currBonus','exploreResets','buildResets',
                                                            'allVertices','nPracticeAttempts','exploreStartTime',
                                                            'buildStartTime','buildFinishTime','targetName','numBlocksExplore']]

df = df_trial_end_reduced_filtered.sort_values(by=['gameID', 'timeAbsolute'])


### Integrate reset data before sending

In [ ]:
# Resets

query = coll.find({"$and":[
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'reset'},
                        {'phase':'build'},
                        {'iterationName':iterationName}]
                     })
df_resets_full = pd.DataFrame(list(query.sort('timeAbsolute')))
df_resets_full_filtered = df_resets_full[df_resets_full.workerId.isin(complete_data_ids)]
df_resets_reduced_filtered = df_resets_full_filtered[['gameID','trialNum','phase','condition','numBlocks']]
pre_reset_blocks = df_resets_reduced_filtered.groupby(
    ['gameID','trialNum','phase','condition'])['numBlocks'].apply(list).reset_index()

# Merge pre-reset blocks with build data
pre_reset_blocks = pre_reset_blocks.rename(columns = {'numBlocks':'preResetBuildBlocks'})

pre_reset_blocks = pre_reset_blocks.fillna(value={'preResetBuildBlocks': 0})
df = df.merge(pre_reset_blocks, on=['gameID', 'trialNum','phase','condition'], how='left')

df2.merge(df1, on['a'], how='left')

# Rename and add totals
df = df.fillna(value={'preResetBuildBlocks': 0})
df = df.rename(columns = {'numBlocks':'finalBuildBlocks'})
df['totalBuildBlocks'] = df['finalBuildBlocks'] + df['preResetBuildBlocks'].apply(np.sum)

In [ ]:
df

In [ ]:
## save out to csv dir, where all the csv's go to live
out_path = os.path.join(csv_dir,'block_silhouette_{}.csv'.format(iterationName))
df_for_analysis.to_csv(out_path)

## Settled Block Data

In [ ]:
print('Loading iteration: ' + iterationName)
query = coll.find({"$and":[
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'settled'},
                        {'iterationName':iterationName}]
                     })

df_settled_full = pd.DataFrame(list(query))


# filter dataframe for complete datasets
df_settled_full_filtered = df_settled_full[df_settled_full.workerId.isin(complete_data_ids)]

print('Loaded ' + str(df_settled_full_filtered.shape[0]) + ' complete sets of settled blocks')
# reduce to crucial information

In [ ]:

df_settled_reduced_filtered = df_settled_full_filtered[['gameID','trialNum','phase','condition',
                                                            'eventType','numBlocks', 'timeAbsolute','timeRelative',
                                                            'normedScore','currBonus','score','incrementalScore','normedIncrementalScore',
                                                            'currBonus','allVertices','targetName','relativePlacementTime','iterationName',
                                                            'blockKind'
                                                           ]]

df_settled_reduced_filtered = df_settled_reduced_filtered.sort_values(by=['gameID', 'timeAbsolute'])

buildstart = df_for_analysis[['gameID','trialNum','buildStartTime','exploreStartTime']]

# copy across time variables that were not saved in the correct place in pilot 3
df_settled_reduced_filtered = df_settled_reduced_filtered.merge(buildstart, on=['gameID', 'trialNum'], how='left')
df_settled_reduced_filtered['timePlaced'] = df_settled_reduced_filtered['timeAbsolute'] - df_settled_reduced_filtered['buildStartTime']
df_settled_reduced_filtered.loc[(df_settled_reduced_filtered.timePlaced < 0),'timePlaced'] = df_settled_reduced_filtered[df_settled_reduced_filtered.timePlaced < 0]['timeAbsolute'] - df_settled_reduced_filtered[df_settled_reduced_filtered.timePlaced < 0]['exploreStartTime']

# 
df_settled_reduced_filtered.loc[(df_settled_reduced_filtered.iterationName == 'pilot4'),'timePlaced'] = df_settled_reduced_filtered[(df_settled_reduced_filtered.iterationName == 'pilot4')]['relativePlacementTime'] 
df_settled_reduced_filtered.loc[(df_settled_reduced_filtered.timePlaced <= 0),'timePlaced'] = 0



In [ ]:
## save out to csv dir, where all the csv's go to live
out_path = os.path.join(csv_dir,'block_silhouette_settled_{}.csv'.format(iterationName))
df_settled_reduced_filtered.to_csv(out_path)

In [ ]:
## load in dataframe
data_path = os.path.join(csv_dir,'block_silhouette_settled_{}.csv'.format(iterationName))
df = pd.read_csv(data_path)
df

## Explore End

In [ ]:
# Ensure one to one gameID and workerId 
# Should only happen if a repeat worker gets through

query = coll.find({"$and":[
                        {'workerId':{'$exists':True}},
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'explore_end'},
                        {"$or":[{'iterationName':'pilot2'},
                                {'iterationName':'pilot3'},
                                {'iterationName':'pilot4'}]},
                        {'trialNum':0}]
                     })

df_explore_end_full = pd.DataFrame(list(query.sort('timeAbsolute')))

assert (np.mean(df_trial_end_full['workerId'].value_counts()) == np.mean(df_trial_end_full['gameID'].value_counts()))

In [ ]:
# get ids of people with trial 15 data
query = coll.find({"$and":[
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'explore_end'},
                        {'iterationName': iterationName},
                        {'trialNum':15}]
                     })
complete_data_df = pd.DataFrame(query)
complete_data_ids = list(complete_data_df['workerId'])

In [ ]:
query = coll.find({"$and":[
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'explore_end'},
                        {'iterationName':iterationName}]
                     })

df_explore_end_full = pd.DataFrame(list(query.sort('timeAbsolute')))



# filter dataframe for complete datasets
df_explore_end_full_filtered = df_explore_end_full[df_explore_end_full.workerId.isin(complete_data_ids)]


# reduce to crucial information
df_explore_end_reduced_filtered = df_explore_end_full_filtered[['gameID','trialNum','phase','condition',
                                                            'eventType','score','normedScore','numBlocks',
                                                            'timeAbsolute','timeRelative',
                                                            'currBonus','exploreResets',
                                                            'allVertices','nPracticeAttempts','exploreStartTime',
                                                            'targetName','numBlocksExplore']]

df_explore = df_explore_end_reduced_filtered.sort_values(by=['gameID', 'timeAbsolute'])

In [ ]:
df_explore

## Integrate reset data before sending

In [ ]:
# Resets

query = coll.find({"$and":[
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'reset'},
                        {'phase':'explore'},
                        {'iterationName':iterationName}]
                     })
df_resets_full = pd.DataFrame(list(query.sort('timeAbsolute')))
df_resets_full_filtered = df_resets_full[df_resets_full.workerId.isin(complete_data_ids)]
df_resets_reduced_filtered = df_resets_full_filtered[['gameID','trialNum','phase','condition','numBlocks']]
pre_reset_blocks = df_resets_reduced_filtered.groupby(
    ['gameID','trialNum','phase','condition'])['numBlocks'].apply(list).reset_index()

# Merge pre-reset blocks with explore data
pre_reset_blocks = pre_reset_blocks.rename(columns = {'numBlocks':'preResetExploreBlocks'})
df_explore = df_explore.merge(pre_reset_blocks, on=['gameID', 'trialNum','phase','condition'], how='left')

# Rename and add totals
df_explore.loc[df_explore.preResetExploreBlocks.isnull(),'preResetExploreBlocks'] = df_explore.preResetExploreBlocks.loc[df_explore.preResetExploreBlocks.isnull()].apply(lambda x:[])
df_explore = df_explore.rename(columns = {'numBlocksExplore':'finalExploreBlocks'})
df_explore['totalExploreBlocks'] = df_explore['finalExploreBlocks'] + df_explore['preResetExploreBlocks'].apply(np.sum)

#df_explore = df_explore.fillna(value={'totalExploreBlocks': 0 })


In [ ]:
df_explore['numAttempts'] = df_explore['preResetExploreBlocks'].apply(len) + 1

In [ ]:
## save out to csv dir, where all the csv's go to live
out_path = os.path.join(csv_dir,'block_silhouette_explore_{}.csv'.format(iterationName))
df_explore.to_csv(out_path)

### Test data saving

In [21]:
iterationName = 'Exp2Pilot2_turk'

In [38]:
query = coll.find({"$and":[
                        #{'condition':{'$ne':'practice'}},
                        {'eventType':'trial_end'},
                        {'iterationName':iterationName}]
                     })
df = pd.DataFrame(list(query.sort('timeAbsolute')))
df

,F1Score,_id,aID,allBlockBodyProperties,allBlockDims,allVertices,blockColor,blockColorID,blockColors,blockFell,...,timeThresholdRed,timeThresholdYellow,timeToBuild,trialNum,version,vertices,workerId,worldHeightUnits,worldScale,worldWidthUnits
0,0.000000,5e277aea5516f33d61c38fb3,3ZDAD0O1T2RFGHC9BCF0HLI5IHRXTB,"[{'id': 6, 'angle': -0.00020941494279422997, '...","[[1, 2], [2, 1], [2, 2], [2, 4], [4, 2]]","[[{'x': 494.95769429406437, 'y': 482.628800642...",#B13B00,0,"[#78878C, #791E94, #6B4623, #FF4A1C, #E85D75, ...",False,...,15000,30000,NaN,NaN,NaN,"[{'x': -247.50000000000017, 'y': 702.428571428...",A30KYQGABO7JER,8,2.2,8
1,0.886779,5e277be15516f33d61c38fc5,3ZDAD0O1T2RFGHC9BCF0HLI5IHRXTB,"[{'id': 21, 'angle': -0.0005891092851112018, '...","[[1, 2], [2, 1], [2, 2], [2, 4], [4, 2]]","[[{'x': 330.0417304873401, 'y': 647.9327819702...",#16BF51,5,"[#78878C, #791E94, #6B4623, #FF4A1C, #E85D75, ...",False,...,15000,30000,59686.0,0.0,82.0,"[{'x': -247.50000000000017, 'y': 702.428571428...",A30KYQGABO7JER,8,2.2,8
2,0.909222,5e277c225516f33d61c38fd6,3ZDAD0O1T2RFGHC9BCF0HLI5IHRXTB,"[{'id': 42, 'angle': 3.576342088246898e-05, 'p...","[[1, 2], [2, 1], [2, 2], [2, 4], [4, 2]]","[[{'x': 330.0336167596471, 'y': 592.8751720610...",#791E94,1,"[#78878C, #791E94, #6B4623, #FF4A1C, #E85D75, ...",False,...,15000,30000,57195.0,1.0,82.0,"[{'x': -247.50000000000017, 'y': 702.428571428...",A30KYQGABO7JER,8,2.2,8
3,0.802682,5e277c635516f33d61c38fe7,3ZDAD0O1T2RFGHC9BCF0HLI5IHRXTB,"[{'id': 63, 'angle': 3.01830121719629e-05, 'po...","[[1, 2], [2, 1], [2, 2], [2, 4], [4, 2]]","[[{'x': 439.85182652444337, 'y': 592.750710004...",#6B4623,2,"[#78878C, #791E94, #6B4623, #FF4A1C, #E85D75, ...",False,...,15000,30000,52807.0,2.0,82.0,"[{'x': -247.50000000000017, 'y': 702.428571428...",A30KYQGABO7JER,8,2.2,8
4,0.546667,5e277ca45516f33d61c38ff2,3ZDAD0O1T2RFGHC9BCF0HLI5IHRXTB,"[{'id': 84, 'angle': 0, 'position': {'x': 605,...","[[1, 2], [2, 1], [2, 2], [2, 4], [4, 2]]","[[{'x': 550, 'y': 592.49681627588}, {'x': 660,...",#E85D75,4,"[#78878C, #791E94, #6B4623, #FF4A1C, #E85D75, ...",False,...,15000,30000,51991.0,3.0,82.0,"[{'x': -247.50000000000017, 'y': 702.428571428...",A30KYQGABO7JER,8,2.2,8
5,0.781899,5e277ce55516f33d61c39003,3ZDAD0O1T2RFGHC9BCF0HLI5IHRXTB,"[{'id': 99, 'angle': -5.241588125727811e-05, '...","[[1, 2], [2, 1], [2, 2], [2, 4], [4, 2]]","[[{'x': 439.9919226234164, 'y': 592.6558597723...",#78878C,0,"[#78878C, #791E94, #6B4623, #FF4A1C, #E85D75, ...",False,...,15000,30000,57652.0,4.0,82.0,"[{'x': -247.50000000000017, 'y': 702.428571428...",A30KYQGABO7JER,8,2.2,8
6,0.934636,5e277d265516f33d61c39018,3ZDAD0O1T2RFGHC9BCF0HLI5IHRXTB,"[{'id': 120, 'angle': 0.00010826062654175446, ...","[[1, 2], [2, 1], [2, 2], [2, 4], [4, 2]]","[[{'x': 494.97211686533115, 'y': 592.608231869...",#E0B300,7,"[#78878C, #791E94, #6B4623, #FF4A1C, #E85D75, ...",False,...,15000,30000,53691.0,5.0,82.0,"[{'x': -247.50000000000017, 'y': 702.428571428...",A30KYQGABO7JER,8,2.2,8
7,0.967528,5e277d675516f33d61c3902f,3ZDAD0O1T2RFGHC9BCF0HLI5IHRXTB,"[{'id': 145, 'angle': 0.0002435762727785026, '...","[[1, 2], [2, 1], [2, 2], [2, 4], [4, 2]]","[[{'x': 275.0404906468102, 'y': 593.4303139339...",#FF4A1C,3,"[#78878C, #791E94, #6B4623, #FF4A1C, #E85D75, ...",False,...,15000,30000,59523.0,6.0,82.0,"[{'x': -247.50000000000017, 'y': 702.428571428...",A30KYQGABO7JER,8,2.2,8


In [49]:
query = coll.find({"$and":[
                        #{'condition':{'$ne':'practice'}},
                        {'eventType':'initial'},
                        {'iterationName':iterationName}]
                     })
df = pd.DataFrame(list(query.sort('timeAbsolute')))
df.columns

Index(['F1Score', '_id', 'aID', 'allBlockDims', 'blockBodyProperties',
       'blockCenterX', 'blockCenterY', 'blockColor', 'blockColorID',
       'blockColors', 'blockDimUnits', 'blockHeight', 'blockKind', 'blockNum',
       'blockOptions', 'blockVertices', 'blockWidth', 'buildDuration',
       'canvasHeight', 'canvasWidth', 'condition', 'currBonus', 'dataType',
       'devMode', 'discreteEnvHeight', 'discreteEnvWidth', 'discreteWorld',
       'eventType', 'floorOptions', 'floorProperties', 'floorY', 'gameID',
       'height_discrete', 'hitID', 'incrementalNormedScoreDiscretePrevious',
       'incrementalScore', 'iterationName', 'menuHeight', 'menuWidth',
       'normedIncrementalScore', 'normedScore', 'normedScoreDiscrete',
       'nullScore', 'nullScoreDiscrete', 'numRepetitions', 'numTargets',
       'numTrials', 'phase', 'points', 'practiceDuration', 'prePostSetSize',
       'prompt', 'randID', 'rawScoreDiscrete', 'relativePlacementTime',
       'repetition', 'scalingFactor', 'sco

In [ ]:
df[['gameID','trialNum','phase','condition',
    'eventType','score','normedScore','numBlocks',
    'timeAbsolute','timeRelative','buildTime',
    'currBonus','allVertices','nPracticeAttempts',
    'buildStartTime','buildFinishTime','targetName',
    'blockColor','blockColorID','numTargets', 'prePostSetSize',
    'numRepetitions','repetition','targetID',
    'bonusThresholdLow','bonusThresholdMid','bonusThresholdHigh',
    'timeThresholdYellow','timeThresholdRed','devMode',
    'timeBonus'
   ]]

In [91]:
query = coll.find({"$and":[
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'trial_end'},
                        {'iterationName':iterationName}]
                     })

df = pd.DataFrame(list(query.sort('gameID')))

# df[['gameID','trialNum','phase','condition',
#     'eventType','score','normedScore','numBlocks',
#     'timeAbsolute','timeRelative','buildTime',
#     'currBonus','allVertices','nPracticeAttempts',
#     'buildStartTime','buildFinishTime','targetName',
#     'blockColor','blockColorID','numTargets', 'prePostSetSize',
#     'numRepetitions','repetition','targetID',
#     'bonusThresholdLow','bonusThresholdMid','bonusThresholdHigh',
#     'timeThresholdYellow','timeThresholdRed','devMode',
#     'timeBonus'
#    ]]

df[['gameID','trialNum','phase','condition','repetition','doNothingRepeats','normedScore','normedScoreDiscrete','targetName','currBonus','timeBonus','score','browser','browserVersion','os']]



,gameID,trialNum,phase,condition,repetition,doNothingRepeats,normedScore,normedScoreDiscrete,targetName,currBonus,timeBonus,score,browser,browserVersion,os
0,0622-493f8d2b-4aa3-4b1c-ad92-f6014b9fc709,0,pre,control,0,0,0.535949,0.535949,hand_selected_004,0.00,0.0,0.00,Chrome,79,Windows
1,0622-493f8d2b-4aa3-4b1c-ad92-f6014b9fc709,1,pre,repeated,0,0,0.526368,0.526368,hand_selected_006,0.00,0.0,0.00,Chrome,79,Windows
2,0622-493f8d2b-4aa3-4b1c-ad92-f6014b9fc709,2,pre,repeated,0,0,0.728734,0.728734,hand_selected_011,0.00,0.0,0.00,Chrome,79,Windows
3,0622-493f8d2b-4aa3-4b1c-ad92-f6014b9fc709,3,pre,repeated,0,0,0.743467,0.743467,hand_selected_009,0.00,0.0,0.00,Chrome,79,Windows
4,0622-493f8d2b-4aa3-4b1c-ad92-f6014b9fc709,4,pre,control,0,0,0.873769,0.873769,hand_selected_012,0.00,0.0,0.00,Chrome,79,Windows
5,0622-493f8d2b-4aa3-4b1c-ad92-f6014b9fc709,5,pre,repeated,0,0,0.565507,0.565507,hand_selected_005,0.00,0.0,0.00,Chrome,79,Windows
6,0622-493f8d2b-4aa3-4b1c-ad92-f6014b9fc709,6,pre,control,0,0,0.753097,0.753097,hand_selected_016,0.00,0.0,0.00,Chrome,79,Windows
7,0622-493f8d2b-4aa3-4b1c-ad92-f6014b9fc709,7,pre,control,0,0,0.689789,0.689789,hand_selected_008,0.00,0.0,0.00,Chrome,79,Windows
8,0622-493f8d2b-4aa3-4b1c-ad92-f6014b9fc709,8,repeated,repeated,1,0,0.813236,0.813236,hand_selected_005,0.00,0.0,0.00,Chrome,79,Windows
9,0622-493f8d2b-4aa3-4b1c-ad92-f6014b9fc709,9,repeated,repeated,1,0,0.643916,0.643916,hand_selected_006,0.00,0.0,0.00,Chrome,79,Windows


In [115]:
# get sequence of trial_end data for all people
# note that these currently lump everyone together, and don't
query = coll.find({"$and":[
                        {'eventType':'survey_data'},
                        {'iterationName':iterationName}]
                     })
df_survey = pd.DataFrame(list(query.sort('absoluteTime')))
df_survey[['workerId','age','comments','difficulty','fun','strategies','inputDevice','sex','score']]

,workerId,age,comments,difficulty,fun,strategies,inputDevice,sex,score
0,A2ZDEERVRN5AMC,28,"Hard! Much harder than I expected! I loved it,...",6,7,Unfortunately I didn't.,Mouse,Male,0.325
1,A17FGZ1I5P9RZA,50,It was vry diificult nut fun I like to use ny ...,6,7,A good base and trying to pay attention to det...,Trackpad,Female,0.090
2,A2LANO898EAYKJ,28,I enjoyed the hit.,4,6,I just saw which piece will be suitable for th...,Mouse,Female,0.070
3,ALLP45O3BDWYM,30,"It was fine, no issues",2,5,redrawing the pattern by imagining that I was ...,Mouse,Female,0.635
4,A1LA6CIGBNDOH9,43,"Frustrating at first, but got the hang of it a...",5,6,Not really. Started from the bottom and would...,Mouse,Male,0.805
5,AY832D29HUURG,25,It was horrible. Unless you got it 100% accura...,7,1,I tried my best at first and realized that unl...,Mouse,Male,0.060
6,A28UGNCW3YMSTH,24,,7,5,,Mouse,Female,0.050
7,ARL7HOWLEHNOP,31,,5,6,just eyeballed it,Mouse,Female,0.405
8,A1ROEDVMTO9Y3X,50,I thought it was fun no problems with it,6,6,Just tried to follow picture the best I could,Mouse,Female,0.000
9,A1VR1XQEQQXYUE,30,,4,5,no,Mouse,Female,0.170


In [103]:
list(df_survey[['workerId','age','comments','difficulty','fun','strategies','inputDevice','sex','score']]['comments'])

['Hard! Much harder than I expected! I loved it, though.',
 'It was vry diificult nut fun I like to use ny brain,',
 'I enjoyed the hit.',
 'It was fine, no issues',
 'Frustrating at first, but got the hang of it after a couple attempts.  Got kinda fun.',
 "It was horrible. Unless you got it 100% accurate, you didn't get a bonus. If you did get a bonus, it was 1-5 cents. And you can't move the blocks after they were placed. ",
 '',
 '',
 'I thought it was fun no problems with it',
 '',
 'I was not able to reset',
 'Everything appeared to run smoothly.',
 "I found the physics really strange. Sometimes it felt like the blocks would fall, but then they didn't. Othertimes the blocks fell when I tried to keep them safe",
 'Interesting and very enjoyable!',
 '']

In [113]:
query = coll.find({"$and":[
                        {'condition':{'$ne':'practice'}},
                        {'eventType':'trial_end'},
                        {'workerId':"A2UCTX06NM6Y02"},
                        {'iterationName':iterationName}]
                     })

df = pd.DataFrame(list(query.sort('gameID')))

# df[['gameID','trialNum','phase','condition',
#     'eventType','score','normedScore','numBlocks',
#     'timeAbsolute','timeRelative','buildTime',
#     'currBonus','allVertices','nPracticeAttempts',
#     'buildStartTime','buildFinishTime','targetName',
#     'blockColor','blockColorID','numTargets', 'prePostSetSize',
#     'numRepetitions','repetition','targetID',
#     'bonusThresholdLow','bonusThresholdMid','bonusThresholdHigh',
#     'timeThresholdYellow','timeThresholdRed','devMode',
#     'timeBonus'
#    ]]

#df[['gameID','trialNum','phase','condition','repetition','doNothingRepeats','normedScore','normedScoreDiscrete','targetName','currBonus','timeBonus','score','browser','browserVersion','os']]

df['score']


0     0.000
1     0.000
2     0.000
3     0.010
4     0.010
5     0.010
6     0.010
7     0.010
8     0.010
9     0.010
10    0.010
11    0.010
12    0.060
13    0.070
14    0.070
15    0.120
16    0.120
17    0.175
18    0.185
19    0.235
20    0.265
21    0.275
22    0.325
23    0.325
Name: score, dtype: float64